In [51]:
import pandas as pd
from config import NDVI_DATA_URL
import os

In [52]:
def get_market_year(row): #make a market year row in datas
    year = row['date'].year
    month = row['date'].month
    if month >= 9:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [53]:
hist = pd.read_csv(f"{NDVI_DATA_URL}/no_cloud/2025.csv").drop("Unnamed: 0", axis=1)

In [54]:
hist["date"] = pd.to_datetime(hist["date"])
hist = hist[hist["date"] >= "2019-09-01"] #year to start, i will take arbitrary last 5 market years
hist["MY"] = hist.apply(get_market_year, axis=1)
hist["MONTH"] = hist["date"].dt.month
hist = hist[~hist["dep"].isin([75, 93])]
current = hist[hist["MY"] == "2024/2025"].copy() #copy current MY
hist = hist[hist["MY"] != "2024/2025"] #remove current MY

In [55]:
meanHist = hist[["dep", "MONTH", "ndvi_mean"]].groupby(["dep", "MONTH"]).mean().reset_index()
meanCurrent = current[["dep", "MONTH", "ndvi_mean"]].groupby(["dep", "MONTH"]).mean().reset_index()

In [59]:
custom_month_order = [9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8] #order month in real MY month series
meanCurrent['MONTHCAT'] = pd.Categorical(meanCurrent['MONTH'], categories=custom_month_order, ordered=True) #convert month column to a categorical type with the custom order
meanCurrent = meanCurrent.sort_values(['dep', 'MONTHCAT']) #sort values with dep then month
meanCurrent['ndvi_mean'] = meanCurrent.groupby('dep')['ndvi_mean'].ffill() #fill nan values with previous month (9 first, if 9 is nan, will still be nan)

In [ ]:
meanCurrent[meanCurrent["ndvi_mean"].isna()] #if not empty, we have missing some data for current year.

In [62]:
meanHist.to_csv("../../YieldModel/data/forecast/currentYear/ndvi_mean_historical.csv")
meanCurrent.to_csv("../../YieldModel/data/forecast/currentYear/ndvi_mean_current.csv")